# Exercise 10 : Gluon Path Integrals

### Function to generate an array of SU(3) matrices and their inverses to use in the monte-carlo process

See Gattringer and Lang page 83. Although they say you should choose random numbers between $(-0.5, 0.5)$ this leads to negative numbers on the diagonal for the generated su3 matrices, these are not closer to the identity for smaller epsilon (as -.999 is generated for epsilon = 0.01 and -.075 is generated for epsilon = 0.5, the first value is obviously further away from 1 even though the epsilon is smaller). Hence smaller epsilon does not give a higher acceptance which is required to tune it for 50%

So I will use random numbers from $(0,0.5)$

In [ ]:
import numpy as np

def generate_su3_array(n, epsilon):

    su3_array = np.empty((2*n, 3, 3), dtype = 'complex128')


    for i in range(n):

        r_random_numbers = np.random.uniform(0, 0.5, (4))
        s_random_numbers = np.random.uniform(0, 0.5, (4))
        t_random_numbers = np.random.uniform(0, 0.5, (4))

        r = np.empty((4))
        s = np.empty((4))
        t = np.empty((4))

        r[1:] = epsilon * r_random_numbers[1:] / np.linalg.norm(r_random_numbers[1:])
        r[0] = np.sign(r_random_numbers[0]) * np.sqrt(1-epsilon**2)

        s[1:] = epsilon * s_random_numbers[1:] / np.linalg.norm(s_random_numbers[1:])
        s[0] = np.sign(s_random_numbers[0]) * np.sqrt(1-epsilon**2)

        t[1:] = epsilon * t_random_numbers[1:] / np.linalg.norm(t_random_numbers[1:])
        t[0] = np.sign(t_random_numbers[0]) * np.sqrt(1-epsilon**2)

        r11 = r[0] + r[3]*1j
        r12 = r[1]*1j + r[2]
        r21 = r[1]*1j - r[2]
        r22 = r[0] - r[3]*1j

        s11 = s[0] + s[3]*1j
        s12 = s[1]*1j + s[2]
        s21 = s[1]*1j - s[2]
        s22 = s[0] - s[3]*1j

        t11 = t[0] + t[3]*1j
        t12 = t[1]*1j + t[2]
        t21 = t[1]*1j - t[2]
        t22 = t[0] - t[3]*1j


        R_su3 = np.array([[r11, r12, 0],
                          [r21, r22, 0],
                          [0, 0, 1]])

        S_su3 = np.array([[s11, 0, s12],
                          [0, 1, 0],
                          [s21, 0, s22]])

        T_su3 = np.array([[1, 0, 0],
                          [0, t11, t12],
                          [0, t21, t22]])


        X_su3 = np.dot(np.dot(R_su3, S_su3), T_su3)


        su3_array[2*i, :, :] = X_su3

        su3_array[2*i+1, :, :] = X_su3.conj().T
        
    return su3_array

#for su3_matrix in su3_array:
#    print(np.round(np.linalg.det(su3_matrix), 18))
#    print(np.round(np.linalg.inv(su3_matrix) - su3_matrix.conj().T, 20))
# Accurate up to 18~ decimal places

### Classes for the lattice and the site at each lattice point.

Site value is just a scalar here as this is a simple gauge simulation. The fundamental data structure of the lattice is a numpy array of site objects which have four links initialised to the identity matrix.

In [ ]:
class site:
    def __init__(self, site_value = 0, link_value = np.identity(3, dtype = 'complex128')):
        
        self.site_value = site_value
        self.link = np.empty((4,3,3), dtype = 'complex128')
        self.link[0] = link_value
        self.link[1] = link_value
        self.link[2] = link_value
        self.link[3] = link_value
        

class lattice:
    def __init__(self, length, n_points, dimension = 4):
        
        self.length = length
        self.n_points  = n_points
        self.dimension = dimension
        self.spacing = self.length / self.n_points
        self.volume = np.repeat(self.n_points, self.dimension)
        self.grid = np.empty(self.volume, dtype = object)
        
        for t in range(self.n_points):
            for x in range(self.n_points):
                for y in range(self.n_points):
                    for z in range(self.n_points):
                        self.grid[t, x, y, z] = site()
        
    def __call__(self, coordinates):
        
        t = int(coordinates[0] % self.n_points)
        x = int(coordinates[1] % self.n_points)
        y = int(coordinates[2] % self.n_points)
        z = int(coordinates[3] % self.n_points)
        
        return self.grid[t, x, y, z]

### Function which computes the staple sum for a given site link

The function takes a coordinate array [t, x, y, z] for the lattice and a direction t = 0, x = 1, y = 2 or z = 3 and computes the sum of staples which include this link. This corresponds to the Wilson action. See Gattringer and Lang page 79 eqn (4.20).

In [ ]:
def wilson_staple_sum(lattice, coordinates, mu):
    
    res = np.zeros((3,3), dtype = 'complex128')
    dimension = lattice.dimension
    
    for nu in range(dimension):
        if nu != mu:
            mu_hat = np.zeros(4, dtype = int)
            nu_hat = np.zeros(4, dtype = int)
            
            mu_hat[mu] = 1
            nu_hat[nu] = 1
            
            res += np.dot(np.dot(lattice(coordinates + mu_hat).link[nu], 
                                 lattice(coordinates + nu_hat).link[mu].conj().T),
                                 lattice(coordinates).link[nu].conj().T)
        
            res += np.dot(np.dot(lattice(coordinates + mu_hat - nu_hat).link[nu].conj().T, 
                                 lattice(coordinates - nu_hat).link[mu].conj().T), 
                                 lattice(coordinates - nu_hat).link[nu])
        
    return res

### Update function for a single gauge link

The function takes a su3 matrix, coordinate array, direction and constant beta and preforms a single montecarlo step for a single gauge link

In [ ]:
def update(lattice, su3_matrix, coordinates, mu, beta):
    
    accept = 1
    
    gauge_link_old = np.copy(lattice(coordinates).link[mu])
    
    lattice(coordinates).link[mu] = np.dot(su3_matrix, gauge_link_old)
    
    diff = lattice(coordinates).link[mu] - gauge_link_old
    
    staple_sum = wilson_staple_sum(lattice = lattice, coordinates = coordinates, mu=mu)
    
    deltaS = -1.0 * beta * np.trace(np.dot(diff, staple_sum)).real
    
    r = np.minimum(1.0, np.exp(-1.0 * deltaS))
    
    if r < np.random.rand():
        
        lattice(coordinates).link[mu] = gauge_link_old
        accept = 0
    
    return accept

### Function to update whole lattice

In [ ]:
def update_lattice(lattice, su3_set, beta):
    
    acceptance = 0
    n_points = lattice.n_points
    dimension = lattice.dimension
    su3_set_length = len(su3_set)
    
    for t in range(n_points):
        for x in range(n_points):
            for y in range(n_points):
                for z in range(n_points):
                    for mu in range(dimension):
                        
                        su3_matrix = su3_set[np.random.randint(0, su3_set_length)]
                        acceptance += update(lattice=lattice, su3_matrix=su3_matrix, 
                                             coordinates=[t,x,y,z], mu=mu, beta=beta)
                            
    return acceptance / (dimension * n_points ** dimension)

### Ensemble Generation

In [ ]:
def markov_chain(lattice, su3_set, beta, n_configs, n_corr, thermalise = True):
    
    acceptance = 0
    ensemble = np.empty(np.append(n_configs, lattice.volume), dtype = object)
    
    
    #Thermalise
    if thermalise == True:
        for i in range(5*n_corr):
            print(i)
            update_lattice(lattice = lattice, su3_set = su3_set, beta = beta)

    for j in range(n_configs*n_corr):
        acceptance += update_lattice(lattice = lattice, su3_set = su3_set, beta = beta)
        
        if j % n_corr == 0:
            
            ensemble_index = int(j/n_corr)
            ensemble[ensemble_index] = lattice.grid
            print(j%n_corr)
            
    acceptance = acceptance / (n_configs*n_corr)       
    
    return ensemble, acceptance    

### Create instance of lattice and thermalise

In [ ]:
import time
lattice = lattice(2, 8)
su3_array = generate_su3_array(50, 0.1)

t1a = time.time()
markov_chain(lattice, su3_array, beta = 5.5, n_configs = 0, n_corr = 50)
t1b = time.time()

print(t1b-t1a)

### Generate ensemble

In [ ]:
t2a = time.time()
ensemble, acceptance = markov_chain(lattice, su3_array, beta = 5.5, n_configs = 200, n_corr = 50, thermalise = False)
t2b = time.time()

print(t2b-t2a)
print(acceptance)

### Wilson loop function for $a \times a$

In [ ]:
def wilson_loop(lattice, coordinates, mu, nu):
    
    dimension = lattice.dimension
    res = np.zeros((3,3), dtype = 'complex128')
    
    mu_hat = np.zeros(4, dtype = int)
    nu_hat = np.zeros(4, dtype = int)
            
    mu_hat[mu] = 1
    nu_hat[nu] = 1
            
    res += np.dot(np.dot(np.dot(lattice(coordinates).link[mu], 
                                lattice(coordinates + mu_hat).link[nu]), 
                                lattice(coordinates + nu_hat).link[mu].conj().T), 
                                lattice(coordinates).link[nu].conj().T)
        
    return np.trace(res).real                     

In [ ]:
def wilson_loop_average(lattice):
    dimension = lattice.dimension
    res = 0
    
    for t in range(n_points):
        for x in range(n_points):
            for y in range(n_points):
                for z in range(n_points):
                    for mu in range(dimension):
                        for nu in range(dimension):
                            if nu != mu:
                                res += wilson_loop(lattice, [t,x,y,z], mu, nu)
        
    return res / ((n_points ** dimension)*(dimension**2)
    